# Algoritmia
## Práctica Obligatoria 3
### Curso 2021 - 2022
###### Programación dinámica
---
 

#### Autores:
* <Guillermo Arcal García>
* <Gonzalo Burgos de la Hera>

---
Resuelva la siguiente práctica.

Importe las librerías que desees
**Recuerda**: 
* Solamente puedes utilizar librerías nativas (https://docs.python.org/es/3.6/library/index.html)
* Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.

In [133]:
# Imports útiles
from copy import deepcopy 

In [134]:
class Video:
    """
    Clase Video. 
    Representa una serie o película.
    """    
    
    def __init__(self, name, size):
        """Crea un objeto de clase Video

        Parameters
        ----------
        name : str
            Nombre de la serie/película
        size : number
            Tamaño en memoria de la serie/película
        """
        self.name = name
        self.size = size
        self.users = {}
    
    def __hash__(self):
        """Genera el valor hash identificativo del vídeo

        Returns
        -------
        int
            Valor hash
        """        
        return hash(self)
    
    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """        
        return "Video: {} - {}".format(self.name, self.size)
    
    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """  
        return "{} - {}".format(self.name, self.size)
       
    def set_users(self, country, users):
        """Dado un pais y un número de usuarios
           almacena para este vídeo la cantidad de espectadores que tiene.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película
        users : int
            Número de espectadores
        """        
        self.users[country] = users
    
    def get_users(self, country):
        """Dado un país, obtiene el número de usuarios.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de espectadores para el país `country`
        """        
        if country in self.users:
            return self.users[country]
        return None

    def tiempo_emision(self, country):
        """Dado un país, obtiene el tiempo de emisión.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de minutos de emisión para el país `country`
        """
        return self.get_users(country) * self.size


In [135]:
class ServidorCache:
    """
    Clase del servidor caché donde se almacenan parte de series/películas.
    """
    
    def __init__(self, identifier, country, capacity):
        """Instancia un Servidor de Caché

        Parameters
        ----------
        identifier : int
            Valor que identifica un servidor.
        country : str
            País donde está el servidor.
        capacity : int
            Cantidad de memoria de almacenamiento disponible.
        """
        self.identifier = identifier
        self.country = country
        self.capacity = capacity
        self.videos_optimizado = []
        
    def __hash__(self):
        """Genera el valor hash identificativo del servidor

        Returns
        -------
        int
            Valor hash
        """    
        return hash(str(self))

    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return "(" + str(self.identifier) + ", " + str(self.country) + ", " + str(self.capacity) + ")"
    
    def __repr__(self):
        """Genera una cadena descriptiva del objeto en colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return "(" + str(self.identifier) + ", " + str(self.country) + ", " + str(self.capacity) + ")"

    def escoger_mejores(self, n, W, tabla, pesos, videos):
        i = n
        j = W

        while (i > 0 and j > 0):
            if(tabla[i][j] != tabla[i-1][j]):
                video = videos[i - 1]
                self.videos_optimizado.append(video)
                j = j - pesos[i - 1]
                i = i - 1
            else:
                i = i - 1

    
    def rellena(self, videos):
        """Dada una colección de videos,
           seleccionar aquellos que se van a almacenar en el servidor.
           Se ha de optimizar para que el tiempo de emisión sea el máximo posible.
           No se pueden partir los vídeos.

        Parameters
        ----------
        videos : collection
            Colección de videos que se quieren almacenar en el servidor.
        """

        tabla = [[0 for i in range(self.capacity + 1)] for n in range(len(videos) + 1)]
        pesos = [x.size for x in videos]
        valores = [x.tiempo_emision(self.country) for x in videos]

        for i in range (1, len(videos) + 1):
            for j in range (1, self.capacity + 1):
                if pesos[i - 1] > j:
                    tabla[i][j] = tabla[i - 1][j]
                else:
                    tabla[i][j] = max(tabla[i - 1][j], tabla[i - 1][j - pesos[i - 1]] + valores[i - 1])

        self.escoger_mejores(len(videos), self.capacity, tabla, pesos, videos)

            
    def tiempo_emision(self):
        """A partir de los datos almacenados
           devolver el tiempo de emisión óptimo del servidor.

        Returns
        -------
        number
            Tiempo de Emision            
        """ 
        t = 0       
        for i in self.videos_optimizado:
            t += i.get_users(self.country) * i.size
        return t


    def almacenados(self):
        """A partir de los datos almacenados
           devolver los objetos vídeo.

        Returns
        -------
        collection
            Colección de videos almacenados en el servidor.
        """     

        # Me parece que esto va a haber que cambiarlo xDDD

        setVideos = []
        for video, tupla in self.dictVideos.items():
            setVideos.append((video, tupla[2]))
        return setVideos
    

In [136]:
class ServidorMaestro:
    """
    Servidor central que gestiona las conexiones entre servidores cache
    """
    
    def __init__(self, servidores, distancias):
        """Instancia el servidor central

        Parameters
        ----------
        servidores : Iterable
            Conjunto de servidores cache disponibles
        distancias : dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """        
        self.servidores = set(servidores)
        self.distancias = distancias        
          
    def get_grafo(self):
        """Devuelve el grafo de distancias recibido

        Returns
        -------
        dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """       
        return self.distancias    
        pass


    def calcula_distancias(self):
        """Calcula las distancias MÍNIMAS entre servidores cache
           y los correspondientes caminos.
        """
        pass


    def distancia(self, origen, destino):
        """
        Devuelve la distancia mmochila[j], left + videos[i].tiempo_emision(self.country)ínima entre dos servidores cache.

        Parameters
        ----------
        origen : ServidorCache
            Servidor de origen
        destino : ServidorCache
            Servidor de destino

        Returns
        -------
        int
            Distancia mínima en milisegundos entre los servidores.
        """        
        pass

    def camino(self, origen, destino):
        """
        Devuelve el camino mínimo entre dos servidores cache.

        Parameters
        ----------
        origen : ServidorCache
            Servidor de origen
        destino : ServidorCache
            Servidor de destino

        Returns
        -------
        list<ServidorCache>
            Lista de servidores para llegar de origen a destino. 
            Se debe incluir al origen y al destino.
        """
        pass

### Pruebas de ejemplo

In [137]:
import unittest
import json

def carga_dataset(data):
    with open(data) as f:
        test_datasets = json.load(f)

    videos = list()
    for v in test_datasets["videos"]:
        v_obj = Video(v["name"], v["size"])
        for c, u in v["users"].items():
            v_obj.set_users(c, u)
        videos.append(v_obj)

    servers = dict()
    for s in test_datasets["servers"]:
        servers[s["country"]] = ServidorCache(s["identifier"], s["country"], s["size"])


    pings = test_datasets["pings"]
    p_ = dict()
    for s in servers.values():
        p_[s] = dict()
        for p in pings[s.country]:
            ping = pings[s.country][p]
            if pings[s.country][p] == -1:
                ping = float("inf")
            p_[s][servers[p]] = ping
    maestro = ServidorMaestro(servers.values(), p_)

    return videos, servers, maestro

class TestBasico(unittest.TestCase):
    
    def test_carga_simple(self):
        
        v, s, m = carga_dataset("toy_PD.json")

        spain = s["Spain"]
        spain.rellena(v)
        self.assertEqual(spain.tiempo_emision(), 55800)
        almacenados = spain.almacenados()
        self.assertEqual(len(almacenados), 3)

        m.calcula_distancias()
        self.assertEqual(m.camino(s["France"], s["Ireland"]), [s["France"], s["Spain"], s["Ireland"]])
        self.assertEqual(m.camino(s["France"], s["Ireland"]), [s["France"], s["Spain"], s["Ireland"]])
        self.assertEqual(m.distancia(s["France"], s["Ireland"]), 200)      
        

In [138]:
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

E
ERROR: test_carga_simple (__main__.TestBasico)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_1880/3849330748.py", line 42, in test_carga_simple
    almacenados = spain.almacenados()
  File "/tmp/ipykernel_1880/2650501680.py", line 122, in almacenados
    for video, tupla in self.dictVideos.items():
AttributeError: 'ServidorCache' object has no attribute 'dictVideos'

----------------------------------------------------------------------
Ran 1 test in 0.005s

FAILED (errors=1)
